In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Reading files from directory
import os
import pickle
 
# Data manipulation & analysis
import pandas as pd
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 500)
import datetime as dt
 
import numpy as np
import scipy

# Visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
 
 # 実行に関係ない警告を無視
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import gc
import json
import math

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold ,StratifiedKFold
from sklearn.linear_model import Ridge ,LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import BayesianRidge


In [ ]:
#実行環境がどこの階層にあるか、パスを検索
import os
print(os.getcwd())

In [ ]:
train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
test = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
submission = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")

In [ ]:
display(train.head())
display(train.describe())
display(train.info())

In [ ]:
display(test.head(10))
display(test.describe())
display(test.info())

In [ ]:
train.columns

In [ ]:
plt.figure(figsize = (10, 5))
sns.kdeplot(train['target'], shade=True)
plt.title("Target Distribution",font="Serif")
plt.show()

In [ ]:
vec = TfidfVectorizer()
train_x = vec.fit_transform(train["excerpt"]).toarray()
test_x = vec.transform(test["excerpt"]).toarray()

In [ ]:
train_y = train["target"].values

In [ ]:
NUM_FOLDS = 10
skf = KFold(n_splits=NUM_FOLDS) # StratifiedKFold
folds = list(skf.split(train_x))

In [ ]:
train_y

In [ ]:
oof = np.zeros(train_y.shape)
pred = np.zeros((test_x.shape[0],))
for idx in range(NUM_FOLDS):
    print("FOLD: ", idx + 1)
    tr_idx, val_idx = folds[idx]
    

    model =Ridge() #Ridge() BayesianRidge(n_iter=300, verbose=True)
    model.fit(train_x[tr_idx], train_y[tr_idx])
    oof[val_idx] =model.predict(train_x[val_idx])
    pred += model.predict(test_x) / NUM_FOLDS
#===============

In [ ]:
mse = mean_squared_error(train_y, oof, squared=False)
print("OOF MSE:",mse)

In [ ]:
submission = test[["id"]].copy()
submission["target"] = pred
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)